In [1]:
%store -r X
%store -r y
%store -r X_test
%store -r y_test
%store -r feature_names

In [2]:
# Import thư viện
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [6]:
import matplotlib.pyplot as plt

# Reshape data
X_2d = X.reshape(-1, X.shape[-1])
y_reshaped = np.repeat(y, 20)

In [3]:
# Khởi tạo mô hình Random Forest
model = RandomForestClassifier(n_estimators=500)

In [8]:
# Huấn luyện mô hình
model.fit(X_2d, y_reshaped)

RandomForestClassifier(n_estimators=500)

In [16]:
# Lấy tầm quan trọng của đặc trưng
feature_importances = model.feature_importances_

# In tầm quan trọng của đặc trưng
print(feature_importances)

[0.0168231  0.01337182 0.01361942 0.01287509 0.01308673 0.01310984
 0.01276929 0.01293364 0.01289002 0.01313857 0.01284673 0.01256
 0.01861947 0.03121628 0.03185601 0.03071387 0.02569574 0.03551281
 0.02466086 0.02561488 0.01566451 0.01973423 0.01680158 0.01857894
 0.01543495 0.0142625  0.0144854  0.01754146 0.01190706 0.00905048
 0.00827862 0.00525087 0.00565982 0.00500901 0.00451929 0.00529244
 0.00437504 0.00453529 0.00479812 0.00478078 0.00530414 0.00423907
 0.00464233 0.00515296 0.0043075  0.00372196 0.00418003 0.00441325
 0.00436903 0.0047341  0.00376127 0.00483213 0.00614055 0.00543121
 0.00460828 0.00548553 0.00552376 0.00702607 0.00536946 0.00560104
 0.00807187 0.00680806 0.00626111 0.00607045 0.00453224 0.00483975
 0.0049     0.00464451 0.00417736 0.00507687 0.00453521 0.00506228
 0.00452566 0.00510815 0.00520481 0.00418501 0.00417109 0.00459659
 0.00550797 0.00609291 0.0053817  0.00529723 0.00499456 0.00525272
 0.00485918 0.00465502 0.00381869 0.00381835 0.00348474 0.0037074

In [50]:
feature_importances.shape

(128,)

In [44]:
# Tạo dataframe gồm tên đặc trưng và tầm quan trọng
feature_importances_df = pd.DataFrame({"feature_name": feature_names, "importance": feature_importances})

# Sắp xếp dataframe theo tầm quan trọng
feature_importances_df = feature_importances_df.sort_values("importance", ascending=False)

# Lấy index của dataframe
indices = feature_importances_df.index

# In index
print(indices)

Index([ 17,  14,  13,  15,  16,  19,  18,  21,  12,  23,
       ...
       109,  50,  45, 112,  89, 114,  90, 107, 106,  88],
      dtype='int64', length=128)


In [45]:
k = 64
top_k_indices = indices[:k]

In [46]:
selected_X = X[:,:,top_k_indices]
selected_X_test = X_test[:,:,top_k_indices]

In [47]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(20, k, 1), padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(5, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 20, 64, 32)        832       
                                                                 
 conv2d_17 (Conv2D)          (None, 20, 64, 32)        25632     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 10, 32, 32)        0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 10, 32, 32)        0         
                                                                 
 conv2d_18 (Conv2D)          (None, 10, 32, 64)        51264     
                                                                 
 conv2d_19 (Conv2D)          (None, 10, 32, 64)        102464    
                                                      

In [48]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(selected_X, y, epochs=10, batch_size=16)

Epoch 1/10
823/823 [==============================] - 40s 46ms/step - loss: 1.0782 - accuracy: 0.5520
Epoch 2/10
823/823 [==============================] - 38s 46ms/step - loss: 0.5399 - accuracy: 0.8034
Epoch 3/10
823/823 [==============================] - 37s 44ms/step - loss: 0.4381 - accuracy: 0.8400
Epoch 4/10
823/823 [==============================] - 38s 46ms/step - loss: 0.3752 - accuracy: 0.8646
Epoch 5/10
823/823 [==============================] - 39s 47ms/step - loss: 0.3573 - accuracy: 0.8697
Epoch 6/10
823/823 [==============================] - 35s 43ms/step - loss: 0.3273 - accuracy: 0.8779
Epoch 7/10
823/823 [==============================] - 34s 42ms/step - loss: 0.2997 - accuracy: 0.8930
Epoch 8/10
823/823 [==============================] - 35s 42ms/step - loss: 0.2784 - accuracy: 0.9004
Epoch 9/10
823/823 [==============================] - 35s 43ms/step - loss: 0.2617 - accuracy: 0.9076
Epoch 10/10
823/823 [==============================] - 35s 43ms/step - loss: 0.235

In [49]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(selected_X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(selected_X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

412/412 [==============================] - 11s 25ms/step - loss: 0.1554 - accuracy: 0.9404
Độ chính xác trên tập kiểm tra: 0.9403766989707947
412/412 [==============================] - 10s 25ms/step
Ma trận nhầm lẫn:
[[2842    0    0    0    0]
 [   0 1961   21   33   19]
 [   1    8 2700    3   96]
 [   0  273   19 1444    4]
 [   0   11  295    2 3434]]
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2842
           1       0.87      0.96      0.91      2034
           2       0.89      0.96      0.92      2808
           3       0.97      0.83      0.90      1740
           4       0.97      0.92      0.94      3742

    accuracy                           0.94     13166
   macro avg       0.94      0.93      0.94     13166
weighted avg       0.94      0.94      0.94     13166

